In [19]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
import torch.nn.functional as F

# Vérifier si CUDA est disponible, sinon utiliser le CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Architecture simplifiée du CNN
class MelanomaCNN(nn.Module):
    def __init__(self):
        super(MelanomaCNN, self).__init__()
        self.conv = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.fc = nn.Linear(16 * 112 * 112, 1)

    def forward(self, x):
        x = F.relu(self.conv(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 16 * 112 * 112)
        x = torch.sigmoid(self.fc(x))
        return x


# Ensemble de données personnalisé
class MelanomaDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None, limit=None):
        self.labels_df = pd.read_csv(csv_file).head(limit)  # Limite à 100 images
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.labels_df.iloc[idx, 0] + ".jpg")
        image = Image.open(img_name)
        label = torch.tensor(self.labels_df.iloc[idx, 1], dtype=torch.float32)
        if self.transform:
            image = self.transform(image)
        return image, label

In [21]:
# Transformations pour les images
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Charger l'ensemble de données
train_dataset = MelanomaDataset(csv_file='../../data/train-labels.csv',  # Ajustez le chemin
                                img_dir='../../data/train-resized/train-resized',      # Ajustez le chemin
                                transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Initialisation du modèle, fonction de perte et optimiseur
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MelanomaCNN().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Entraînement du modèle
epochs = 10
for epoch in range(epochs):
    for data, target in train_loader:
        data, target = data.to(device).float(), target.to(device).float().unsqueeze(1)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

# Sauvegarder le modèle
torch.save(model.state_dict(), 'path_to_save_model.pth')

Epoch 1/10, Loss: 0.008033929392695427
Epoch 2/10, Loss: 0.0004863036738242954
Epoch 3/10, Loss: 0.017347946763038635
Epoch 4/10, Loss: 0.005256232805550098
Epoch 5/10, Loss: 0.0037439081352204084
Epoch 6/10, Loss: 0.003921271301805973
Epoch 7/10, Loss: 0.00025818555150181055
Epoch 8/10, Loss: 0.9824573993682861
Epoch 9/10, Loss: 0.021516110748052597
Epoch 10/10, Loss: 0.003237965516746044


In [22]:
# Sauvegarder le modèle
torch.save(model.state_dict(), 'melanoma_model.pth')  # Ajustez le chemin
